In [77]:
import sys
sys.path.append("..")
import d2lzh_pytorch as d2l
import torch
import numpy as np
import torchvision

In [78]:
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

In [79]:
num_inputs = 784#像素数
num_outputs = 10#分类数

W = torch.tensor(np.random.normal(0, 0.01, (num_inputs, num_outputs)), dtype=torch.float,requires_grad=True)
b = torch.zeros(num_outputs, dtype=torch.float,requires_grad=True)


In [80]:
def softmax(X):
    XX=X.exp()
    participation=XX.sum(dim=1,keepdim=True)
    return XX / participation

In [81]:
#Try
x=torch.randn(2,5)
print(x)
x=softmax(x)
print(x,x.sum(dim=1,keepdim=True))

tensor([[-1.1235, -0.5948,  1.1606,  1.1108,  1.3203],
        [ 0.9074,  0.0396,  1.1916, -0.2211,  1.2097]])
tensor([[0.0300, 0.0508, 0.2942, 0.2799, 0.3451],
        [0.2260, 0.0949, 0.3003, 0.0731, 0.3058]]) tensor([[1.],
        [1.]])


In [82]:
def net(X):
    return softmax(torch.mm(X.view((-1, num_inputs)), W) + b)
#X矩阵：行数=样本数，列数=像素数
#O_hat=XW+b(b是行向量)

view(参数a,参数b,…)，其中，如果某个参数为-1，则表示该维度取决于其它维度，由Pytorch自己补充。<p>
view(-1)表示将Tensor转为一维Tensor。

#### <mark> 通过gether实现crossEntropy函数，作为损失函数 <mark>
https://blog.csdn.net/weixin_42899627/article/details/122816250

In [83]:
y_hat = torch.tensor([[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]])
y = torch.LongTensor([0, 2])
y_hat.gather(1, y.view(-1, 1))


tensor([[0.1000],
        [0.5000]])

In [84]:
def cross_entropy(y_hat,y):
    return -torch.log(y_hat.gather(1,y.view(-1,1)))

argmax(dim=1)返回矩阵每行中最大值，按列组装成向量

In [85]:
def accuracy(y_hat, y):
    return (y_hat.argmax(dim=1) == y).float().mean().item()


mean()将tensor转换成算术平均的零维向量，item将零维向量转换成python数

In [86]:
print(accuracy(y_hat, y))


0.5


In [87]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用。该函数将被逐步改进：它的完整实现将在“图像增广”一节中描述
def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0.0, 0
    for X, y in data_iter:
        acc_sum += (net(X).argmax(dim=1) == y).float().sum().item()
        n += y.shape[0]    #行数
    return acc_sum / n


In [88]:
evaluate_accuracy(test_iter, net)

0.0798

In [89]:
num_epochs, lr = 5, 0.1

# 本函数已保存在d2lzh包中方便以后使用
def train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size,
              params=None, lr=None, optimizer=None):
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n = 0.0, 0.0, 0
        for X, y in train_iter:
            y_hat = net(X)
            l = loss(y_hat, y).sum()

            # 梯度清零
            if optimizer is not None:
                optimizer.zero_grad()
            elif params is not None and params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()

            l.backward()
            if optimizer is None:
                d2l.sgd(params, lr, batch_size)
            else:
                optimizer.step()  # “softmax回归的简洁实现”一节将用到


            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().item()
            n += y.shape[0]
        test_acc = evaluate_accuracy(test_iter, net) #前提是W,b是全局变量，否则需要将W,b传入net中  #net本质上是函数对象
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc))

train_ch3(net, train_iter, test_iter, cross_entropy, num_epochs, batch_size, [W, b], lr)


epoch 1, loss 0.7854, train acc 0.750, test acc 0.792
epoch 2, loss 0.5704, train acc 0.814, test acc 0.812
epoch 3, loss 0.5250, train acc 0.825, test acc 0.816
epoch 4, loss 0.5006, train acc 0.833, test acc 0.822
epoch 5, loss 0.4848, train acc 0.837, test acc 0.829


In [90]:
X, y = iter(test_iter).next()

true_labels = d2l.get_fashion_mnist_labels(y.numpy())
pred_labels = d2l.get_fashion_mnist_labels(net(X).argmax(dim=1).numpy())
titles = [true + '\n' + pred for true, pred in zip(true_labels, pred_labels)]

d2l.show_fashion_mnist(X[0:9], titles[0:9])


AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute 'next'